In [7]:
from gemini.gemini import Gemini
from gemini.helpers import poloniex as px
from gemini.helpers.analyze import analyze_mpl, analyze_bokeh
from src.data.poloniex_data_proc import get_historical_data_from_file
import bokeh


def logic(algo, data):
    """
    Main algorithm method, which will be called every tick.

    :param algo: Gemini object with account & positions
    :param data: History for current day
    """

    if len(data) < 2:
        # Skip short history
        return

    today = data.iloc[-1]  # Current candle
    yesterday = data.iloc[-2]  # Previous candle

    # close positions
    if len(algo.account.positions) > 0:
        if today['close'] < yesterday['close']:
            exit_price = today['close']
            for position in algo.account.positions:
                if position.type_ == 'Long':
                    algo.account.close_position(position, 1, exit_price)

    # open positions
    elif today['close'] > yesterday['close']:
        risk = 0.9
        entry_price = today['close']
        entry_capital = algo.account.buying_power * risk
        if entry_capital >= 0.00001:
            algo.account.enter_position('Long', entry_capital, entry_price)

Available cryptocurrency pairs:

Bitcoin:
AMP, ARDR, BCH, BCN, BCY, BELA, BLK, BTCD, BTM, BTS, BURST, CLAM, CVC, DASH, DCR, DGB, DOGE, EMC2, 
ETC, ETH, EXP, FCT, FLDC, FLO, GAME, GAS, GNO, GNT, GRC, HUC, LBC, LSK, LTC, MAID, NAV, NEOS, NMC, NXC, 
NXT, OMG, OMNI, PASC, PINK, POT, PPC, RADS, REP, RIC, SBD, SC, STEEM, STORJ, STR, STRAT, SYS, VIA, VRC, 
VTC, XBC, XCP, XEM, XMR, XPM, XRP, XVC, ZEC, ZRX

Ethereum:
BCH, CVC, ETC, GAS, GNO, GNT, LSK, OMG, REP, STEEM, ZEC, ZRX

USD Tether:
BCH, BTC, DASH, ETC, ETH, LTC, NXT, REP, STR, XMR, XRP, ZEC

Monero
BCN, BLK, BTCD, DASH, LTC, MAID, NXT, ZEC

In [13]:
# get data from file stored on disk
start_date = '2017-01-04 0:00:00'
end_date = '2017-12-31 23:59:59'
base_currency = 'BTC'
other_currency = 'XRP'
resample_frequency = '1D'

df = get_historical_data_from_file(base_currency, other_currency, resample_frequency, start_date, end_date)
print(df['open', 'high', 'low', 'close'].head()) # ignore other columns apart from ohlc
print(df.describe())

KeyError: ('open', 'high', 'low', 'close')

In [9]:
# Algorithm settings
sim_params = {
    'capital_base': 10,      # initial capital in BTC
    'fee': {
        'Long': 0.0015,      # fee settings for Long
        'Short': 0.0015,     # fee settings for Short
    },
    'data_frequency': '1D'    # Time frame to use (see /helpers/timeframe_resampler.py for more info
}
gemini = Gemini(logic=logic, sim_params=sim_params, analyze=analyze_bokeh)

In [10]:
# start backtesting custom logic with 10 (BTC) intital capital
gemini.run(df, show_trades=True)

================ Results (freq 1D) =================

-------------- Benchmark ---------------
Capital      : 10.00
Final Equity : 250.88
Net profit   : 240.88 (+2408.76%)
Max Drawdown : -94.15%
--------------- Strategy ---------------
Capital      : 10.00
Final Equity : 251.85
Net profit   : 241.85 (+2418.45%)
Max Drawdown : -63.77%
Fees paid    : 13.70
-------------------- Statistics --------------------
                        Long      Short        All
Success rate %      :    30.3371     0.0000    30.3371
Avg Win / trade     :    14.4967     0.0000    14.4967
Avg Loss / trade    :    -2.4181     0.0000    -2.4181
Expected value      :     2.7134     0.0000     2.7134
Open                :    89.0000     0.0000    89.0000
Closed              :    89.0000     0.0000    89.0000
Total Trades        :   178.0000     0.0000   178.0000
----------------------------------------------------


Loading BokehJS ...

,open,high,low,close,volume,date,price,base_equity,equity,benchmark_period_return,benchmark_max_drawdown,algorithm_period_return,returns,max_drawdown,ending_value,alpha,beta,sharpe
index,,,,,,,,,,,,,,,,,,
2017-01-04,0.00000618,0.00000626,0.00000563,0.00000571,,2017-01-04,0.00000571,10.00000000,10.00000000,0.00000000,nan,0.00000000,nan,nan,0,0,0,0
2017-01-05,0.00000574,0.00000655,0.00000420,0.00000620,,2017-01-05,0.00000620,10.85814361,10.00000000,0.08581436,0.00000000,0.00000000,0.00000000,0.00000000,0,0,0,0
2017-01-06,0.00000620,0.00000738,0.00000600,0.00000707,,2017-01-06,0.00000707,12.38178634,11.24739129,0.23817863,0.00000000,0.12473913,0.12473913,0.00000000,0,0,0,0
2017-01-07,0.00000705,0.00000759,0.00000687,0.00000707,,2017-01-07,0.00000707,12.38178634,11.24739129,0.23817863,0.00000000,0.12473913,0.00000000,0.00000000,0,0,0,0
2017-01-08,0.00000706,0.00000720,0.00000667,0.00000686,,2017-01-08,0.00000686,12.01401051,10.94304347,0.20140105,0.00000000,0.09430435,-0.02705941,0.00000000,0,0,0,0
2017-01-09,0.00000685,0.00000709,0.00000672,0.00000685,,2017-01-09,0.00000685,11.99649737,10.92813042,0.19964974,-0.02970297,0.09281304,-0.00136279,-0.02705941,0,0,0,0
2017-01-10,0.00000683,0.00000780,0.00000680,0.00000750,,2017-01-10,0.00000750,13.13485114,10.92813042,0.31348511,-0.03111740,0.09281304,0.00000000,-0.02838533,0,0,0,0
2017-01-11,0.00000750,0.00000850,0.00000708,0.00000829,,2017-01-11,0.00000829,14.51838879,11.94664815,0.45183888,-0.03111740,0.19466481,0.09320146,-0.02838533,0,0,0,0
2017-01-12,0.00000829,0.00000920,0.00000749,0.00000794,,2017-01-12,0.00000794,13.90542907,11.48888737,0.39054291,-0.03111740,0.14888874,-0.03831709,-0.02838533,0,0,0,0
